# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.234579546853                   -0.50    8.0         
  2   -7.250075998611       -1.81       -1.39    1.0   9.09ms
  3   -7.250938524686       -3.06       -1.97    1.0   9.28ms
  4   -7.250961904197       -4.63       -1.88    2.0   11.7ms
  5   -7.251331910910       -3.43       -2.69    1.0   9.32ms
  6   -7.251338073287       -5.21       -3.12    2.0   11.4ms
  7   -7.251338687012       -6.21       -3.58    1.0   9.73ms
  8   -7.251338778585       -7.04       -3.91    2.0   11.7ms
  9   -7.251338796857       -7.74       -4.45    1.0   9.65ms
 10   -7.251338798523       -8.78       -4.96    2.0   11.8ms
 11   -7.251338798687       -9.79       -5.30    2.0   12.1ms
 12   -7.251338798702      -10.82       -5.79    1.0   9.91ms
 13   -7.251338798704      -11.67       -6.20    1.0   10.1ms
 14   -7.251338798704      -12.17       -6.35    3.0   13.0ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.050666586563343594
[ Info: Arnoldi iteration step 2: normres = 0.6012211321239032
[ Info: Arnoldi iteration step 3: normres = 0.8891855789965296
[ Info: Arnoldi iteration step 4: normres = 0.2818432104934659
[ Info: Arnoldi iteration step 5: normres = 0.43208996587333004
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (2.84e-02, 6.33e-02, 3.52e-01, 2.34e-01, 5.72e-02)
[ Info: Arnoldi iteration step 6: normres = 0.3654710854764077
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (9.29e-03, 2.26e-01, 2.49e-01, 1.18e-01, 2.98e-02)
[ Info: Arnoldi iteration step 7: normres = 0.0890302686473067
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (4.13e-04, 2.15e-02, 1.21e-02, 4.16e-02, 6.80e-02)
[ Info: Arnoldi iteration step 8: normres = 0.1206900608372971
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (2.18e-0